# Majorana nanowire conductance, band structure, phase diagram, and wave function

In [ ]:
import numpy as np
import kwant
from scipy.constants import hbar, m_e, eV, physical_constants
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.size'] = 20

meV = eV * 1e-3  # one meV in J
mu_B = physical_constants['Bohr magneton'][0] / meV  # Bohr magneton in meV
g = 50
m = 0.015 * m_e  # the effective mass in kg
a = 5  # the lattice spacing in nm
t = (hbar ** 2 / (2 * m * a ** 2)) * (1e18 / meV)  # meV

s0 = np.array([[1., 0.], [0., 1.]])
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1j], [1j, 0.]])
sz = np.array([[1., 0.], [0., -1.]])

# Tunnel spectroscopy
# the system will consist of three subsystems
# We apply a magnetic field in all parts of the system
def onsite_sc(site, mu_sc, B_x, delta):
    return ((2 * t - mu_sc) * np.kron(s0, sz) 
            + (0.5 * B_x * g * mu_B) * np.kron(sx, s0) 
            + delta * np.kron(s0, sx))


def onsite_normal(site, mu_l, B_x):
    return ((2 * t - mu_l) * np.kron(s0, sz) 
            + (0.5 * B_x * g * mu_B) * np.kron(sx, s0))


def onsite_barrier(site, mu_l, B_x, Vbarrier):
    return ((2 * t - mu_l + Vbarrier) * np.kron(s0, sz) 
            + (0.5 * B_x * g * mu_B) * np.kron(sx, s0))


# The hopping is the same in all subsystems. There is normal hopping and
# spin orbit interaction.
def hop(site1, site2, alpha):
    return -t * np.kron(s0, sz) + 0.5j * alpha / a * np.kron(sy, sz)


def make_system_spectroscopy(t=t, a=a):
    lat = kwant.lattice.chain(a=a, norbs=4)
    sys = kwant.Builder()

    # The first subsystem just consists of the tunnel barrier (one site with a
    # potential)
    sys[lat(0)] = onsite_barrier

    # The second subsystem is a normal lead
    # The translational symmetry makes it semiinfinite.
    symmetry = kwant.TranslationalSymmetry(*lat.prim_vecs)
    lead_normal = kwant.Builder(symmetry, conservation_law=-np.kron(s0, sz))
    # Define a unit cell (in this case the unitcell consists of a single site)
    lead_normal[lat(0)] = onsite_normal
    # Define the hopping between unitcells
    lead_normal[lat(1), lat(0)] = hop

    # The third subsystem is a superconducting lead. A Majorana bound state
    # can arise at the edge of this system.
    lead_sc = kwant.Builder(symmetry)
    # Again: Define a unitcell
    lead_sc[lat(0)] = onsite_sc
    # Again define hopping between unitcells
    lead_sc[lat(1), lat(0)] = hop

    # Create a connection between the first subsystem (the tunnelbarrier, system name: sys)
    # and the other two subsystems (the normal and the superconducting lead)
    sys.attach_lead(lead_normal)
    sys.attach_lead(lead_sc)

    sys = sys.finalized()

    return sys


def make_majorana_nanowire(t=t, a=a, L=100, as_lead=False, finalize=True, n_cells_in_unit_cell=1):
    lat = kwant.lattice.chain(a=a, norbs=4)
    symmetry = kwant.TranslationalSymmetry([n_cells_in_unit_cell * a]) if as_lead else None
    sys = kwant.Builder(symmetry)
    sys[(lat(i) for i in range(L//a))] = onsite_sc
    sys[lat.neighbors()] = hop
    if finalize:
        sys = sys.finalized()
    return sys


def tunnel_spectroscopy(params):
    sys = make_system_spectroscopy()
    Es = np.linspace(-1.2*params['delta'], 1.2*params['delta'], 51) + 1e-5
    Gs = []
    for E in Es:
        # This function returns the scatering matrix (at the energy given by the second argument)
        # of the system. The scattering matrix holds the information about
        # conductivity.
        smat = kwant.smatrix(
            sys, energy=E, params=params)
        Andreev_conductance = (smat.submatrix((0, 0), (0, 0)).shape[0] -
                smat.transmission((0, 0), (0, 0)) +
                smat.transmission((0, 1), (0, 0)))
        Gs.append(Andreev_conductance)

    return Es, Gs


def majorana_size_from_modes(lead, params):
    """Find the slowest decaying (evanescent) mode and returns the
    Majorana size.

    It uses an adapted version of the function kwant.physics.leads.modes,
    in such a way that it returns the eigenvalues of the translation operator
    (lamdba = |r|*e^ik). The imaginary part of the wavevector k, is the part
    that makes it decay. The inverse of this Im(k) is the size of a Majorana
    bound state. The norm of the eigenvalue that is closest to one is the
    slowest decaying mode. Also called decay length.

    Parameters:
    -----------
    lead : kwant.builder.InfiniteSystem object
        The finalized infinite system.

    Returns:
    --------
    majorana_length : float
        The length of the Majorana.
    """
    h = lead.cell_hamiltonian(params=params)
    t = lead.inter_cell_hopping(params=params)
    a, b = kwant.physics.leads.setup_linsys(h, t, 1e6, None).eigenproblem
    ev = kwant.physics.leads.unified_eigenproblem(a, b, tol=1e6)[0]    
    norm = ev * ev.conj()
    idx = np.abs(norm - 1).argmin()
    a = lead.sites[0].family.prim_vecs[0]
    majorana_length = np.abs(a / np.log(ev[idx]).real)
    return float(majorana_length)


def plot_spectroscopy(mu_l, mu_sc, B_x, alpha, delta, Vbarrier):
    fig, ax = plt.subplots()
    params = dict(mu_l=mu_l, mu_sc=mu_sc, B_x=B_x, alpha=alpha, delta=delta, Vbarrier=Vbarrier)
    # Trivial, because the magnetic field is zero (third argument)
    Es, Gs = tunnel_spectroscopy(dict(params, B_x=0))
    ax.plot(Es, Gs, 'k-', label='trivial')

    # Non-trivial
    Es, Gs = tunnel_spectroscopy(params)
    ax.plot(Es, Gs, 'r-', label='non trivial')

    ax.set_title('Tunnel spectroscopy')
    ax.set_xlabel('Bias Voltage')
    ax.set_xlim([-1.2 * delta, 1.2 * delta])
    ax.set_ylabel('$G/G_0$')
    ax.set_ylim([-0.0, 2.1])

    ax.legend(fontsize=7)

    return fig


def plot_bandstructure(mu_l, mu_sc, B_x, alpha, delta, show_gap):
    fig, ax = plt.subplots()
    params = dict(mu_l=mu_l, mu_sc=mu_sc, B_x=B_x, alpha=alpha, delta=delta, Vbarrier=None)
    ks = np.linspace(-np.pi/4, np.pi/4, 201)
    lead = make_system_spectroscopy().leads[1]
    bands = kwant.physics.Bands(lead, params=params)
    Es = [bands(k) for k in ks]
    ax.plot(ks, Es, 'k-')

    ax.set_xlabel(r'$k_x \; (\textrm{nm}^{-1})$')
    ax.set_ylabel(r'$E \; (\textrm{meV})$')
    ax.set_title(rf'\textrm{{Topological band structure}}')
    ax.set_xticks([-np.pi/4, 0, np.pi/4])
    ax.set_xticklabels(['$-\pi/4$', '$0$', '$\pi/4$'])
    yticks = [-2, 0, 2]
    ax.set_yticks(yticks)
    ax.set_yticklabels([f'${y}$' for y in yticks])
    ax.set_ylim(yticks[0], yticks[-1])
    if show_gap:
        E_gap = np.abs(Es).min()
        x_right = ks[-1] / 2
        x_left = ks[0]
        ax.hlines(E_gap, x_left, ks[-1], linestyles='--', colors='r')
        ax.hlines(0, x_left, x_right, linestyles='--', colors='k')
        ax.hlines(-E_gap, x_left, ks[-1], linestyles='--', colors='r')
        ax.annotate(r'$2E_\textrm{gap}$', (x_right + 0.05, 0), va='center', ha='left')
        props = dict(head_width=0.1, head_length=0.15, overhang=0.4, color='k', length_includes_head=True)
        ax.arrow(x_right, 0, dx=0, dy=E_gap, **props)
        ax.arrow(x_right, 0, dx=0, dy=-E_gap, **props)
    return fig


def plot_wavefunction(mu_sc=10, B_x=1, alpha=50, delta=1, L=300, a=5, calc_xi_M=False):
    params = dict(mu_sc=mu_sc, B_x=B_x, alpha=alpha, delta=delta, L=L)
    syst = make_majorana_nanowire(L=L, a=a)
    ham = syst.hamiltonian_submatrix(params=dict(params))
    energies, wfs = np.linalg.eigh(ham)
    i_min = np.abs(energies).argmin()
    rho = kwant.operator.Density(syst)
    wf = rho(wfs[:, i_min])
    fig, ax = plt.subplots()
    ax.plot(wf)
    ax.set_xlabel(r'$L \; \textrm{(nm)}$')
    ax.set_ylabel(r'$\left| \Psi_\textrm{M} \right|^2$')
    xticks = [0, L/a//2, L/a]
    ax.set_xticks(xticks)
    ax.set_xticklabels([f'${a*x}$' for x in xticks])
    yticks = [0, round(wf.max()+5e-3, 2)]
    ax.set_yticks(yticks)
    ax.set_yticklabels([f'${y}$' for y in yticks])

    title = rf'\textrm{{Majorana wave function}}'
    if calc_xi_M:
        lead = make_system_spectroscopy(t=t, a=a).leads[1]
        xi_M = majorana_size_from_modes(lead, params)
        title += fr', $\xi_{{\textrm{{M}}}}={xi_M:.1f} \; \textrm{{nm}}$'
    ax.set_title(title)
    return fig

def plot_phase_transition(delta):
    fig, ax = plt.subplots()
    ax.set_title(r'$\textrm{Majorana phase diagram}$')
    ax.set_xlabel(r'$B_x \; \textrm{(T)}$')
    ax.set_ylabel(r'$\mu \; \textrm{(meV)}$')
    B_xs = np.linspace(0, 2, 2000)
    ax.set_xlim(0, B_xs[-1])
    def phase_transition(B_x, delta):
        E_z = .5 * g * mu_B * B_x
        mu = np.sqrt(E_z**2 - delta**2)
        return np.vstack([mu, -mu]).T
    boundary = phase_transition(B_xs, delta)
    ax.plot(B_xs, boundary, c='C1')
    trivial = np.isnan(boundary[:, 0])
    ax.fill_between(B_xs[~trivial], boundary[~trivial, 0], boundary[~trivial, 1], color='C2', label='topological')
    ax.fill_between([0, 1e-3], 0.0, 1e-3, color='white', label='trivial')
    ax.legend(loc='upper left', facecolor='grey', framealpha=0.2)
    return fig

In [ ]:
from ipywidgets import interact, FloatSlider, Checkbox, IntSlider
interact(plot_wavefunction, 
         mu_sc=FloatSlider(min=0, max=25, step=0.1, value=0, description=r"$\mu_{SC}$ in meV"), 
         B_x=FloatSlider(min=0, max=10, step=0.05, value=0.7, description=r"Magnetic field $(B)$ in T"),
         alpha=FloatSlider(min=0, max=100, step=0.5, value=50, description=r"SO coupling $(\alpha)$ in meV$\cdot$nm"),
         delta=FloatSlider(min=0, max=0.5, step=0.05, value=0.50, description=r"SC order param. ($\Delta$) in meV"),
         L=IntSlider(min=50, max=1000, step=50, value=300, description='Length of nanowire in nm.'),
         a=IntSlider(min=1, max=10, step=1, value=1, description='Lattice contant in nm.'),
         calc_xi_M=Checkbox(description=r'Fit $\xi_M$', value=False)
);

All units are in nm, T, and meV.

In [ ]:
from ipywidgets import interact, FloatSlider, Checkbox
interact(plot_bandstructure, 
         mu_l=FloatSlider(min=0, max=50, step=0.1, value=25, description=r"$\mu_l$ in meV"),
         mu_sc=FloatSlider(min=0, max=25, step=0.1, value=0, description=r"$\mu_{SC}$ in meV"), 
         B_x=FloatSlider(min=0, max=10, step=0.05, value=0.7, description=r"Magnetic field $(B)$ in T"),
         alpha=FloatSlider(min=0, max=100, step=0.5, value=50, description=r"SO coupling $(\alpha)$ in meV$\cdot$nm"),
         delta=FloatSlider(min=0, max=0.5, step=0.05, value=0.50, description=r"SC order param. ($\Delta$) in meV"),
         show_gap=Checkbox(description='Show E_gap', value=True));

In [ ]:
from ipywidgets import interact, FloatSlider
interact(plot_spectroscopy, 
         mu_l=FloatSlider(min=0, max=50, step=0.1, value=15, description=r"$\mu_l$ in meV"),
         mu_sc=FloatSlider(min=0, max=25, step=0.1, value=0, description=r"$\mu_{SC}$ in meV"), 
         B_x=FloatSlider(min=0, max=10, step=0.05, value=0.5, description=r"Magnetic field $(B)$ in T"),
         alpha=FloatSlider(min=0, max=50, step=0.5, value=20, description=r"SO coupling $(\alpha)$ in meV$\cdot$nm"),
         delta=FloatSlider(min=0, max=0.5, step=0.05, value=0.25, description=r"SC order param. ($\Delta$) in meV"),
         Vbarrier=FloatSlider(min=0, max=100, step=0.25, description=r"$V_{barrier}$ in meV"));

In [ ]:
from ipywidgets import interact, FloatSlider
interact(plot_phase_transition, 
         delta=FloatSlider(min=0, max=2, step=0.05, value=0.25, description=r"SC order param. ($\Delta$) in meV"),);

In [ ]:
# Saving some plots for a presentation
f = kwant.plot(make_majorana_nanowire(L=100), show=False)
ax, = f.get_axes()
plt.axis('off')
f.savefig('finite_system.svg', bbox_inches='tight')
for i in range(1, 5):
    f = kwant.plot(make_majorana_nanowire(L=5*i, as_lead=True,
        finalize=False, n_cells_in_unit_cell=i), show=False)
    plt.axis('off')
    f.savefig(f'infinite_system_{i}.svg', bbox_inches='tight')
f = plot_wavefunction(0, 0.7, 50, 0.50, 300, 1, True)
f.savefig('wf.svg', bbox_inches='tight')
f = plot_bandstructure(25, 0, 0.7, 50, 0.50, True)
f.savefig('band_structure.svg', bbox_inches='tight')
f = plot_phase_transition(1)
f.savefig('phase_diagram.svg', bbox_inches='tight')